# INÍCIO

---
PONTIFÍCIA UNIVERSIDADE CATÓLICA DE MINAS GERAIS<br>
CURSO: Ciencia de dados e big data<br>
DISCIPLINA: Integracao e processamentos de fluxos continuos de dados <br>
* ALUNO: Pedro Penido Gama<br>
* MATRÍCULA: 149820<br>
* PROFESSOR: VICTOR SALES SILVA<br>
* ATIVIDADE: ATIVIDADE DE MANIPULAÇÃO DE DADOS USANDO LINGUAGEM PYTHON/SPARK<br>
---


## CONFIGURAÇÃO DO AMBIENTE

### INSTALAÇÃO DAS BIBLIOTECAS

#### BIBLIOTECA PARA MANIPULAÇÃO DE DADOS NO AZURE STORAGE ACCOUNT

In [1]:
pip install azure-storage-blob

     |████████████████████████████████| 178 kB 1.5 MB/s eta 0:00:01
ERROR: azureml-core 1.42.0 has requirement azure-core<=1.22.1, but you'll have azure-core 1.24.2 which is incompatible.
ERROR: azureml-contrib-notebook 1.42.0 has requirement nbconvert<6, but you'll have nbconvert 6.5.0 which is incompatible.
ERROR: azure-cli 2.37.0 has requirement azure-graphrbac~=0.60.0, but you'll have azure-graphrbac 0.61.1 which is incompatible.
ERROR: azure-cli 2.37.0 has requirement azure-mgmt-authorization~=0.61.0, but you'll have azure-mgmt-authorization 2.0.0 which is incompatible.
ERROR: azure-cli 2.37.0 has requirement azure-mgmt-containerregistry==8.2.0, but you'll have azure-mgmt-containerregistry 9.1.0 which is incompatible.
ERROR: azure-cli 2.37.0 has requirement azure-mgmt-resource==21.1.0b1, but you'll have azure-mgmt-resource 21.0.0 which is incompatible.
ERROR: azure-cli-core 2.37.0 has requirement argcomplete~=1.8, but you'll have argcomplete 2.0.0 which is incompatible.
ERROR: azu

#### BIBLIOTECA PARA MANIPULAR DADOS USANDO SPARK (PYTHON)

In [2]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


#### BIBLIOTECA PARA USAR A API DO SERVIÇO COGNITIVE SERVICES (ANÁLISE DE TEXTOS) DO MICROSOFT AZURE

In [3]:
pip install azure-ai-textanalytics

     |████████████████████████████████| 153 kB 1.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### IMPORTAÇÃO DAS BIBLIOTECAS

In [5]:
import requests                                                   # Requisições WEB
from azure.storage.blob import BlobServiceClient, BlobClient      # Manipulação de arquivos no Azure Storage Blob
from pyspark.sql import SQLContext, SparkSession, functions as F  # Criar e manipular aplicações/funções em Spark
from pyspark.sql.types import *                                   # Manipular tipos de dados mapeados pelo Spark
from azure.ai.textanalytics import TextAnalyticsClient            # Importar cleinte de análise de texto
from azure.core.credentials import AzureKeyCredential             # Importar módulo para criação de credenciais
import pandas                                                     # Manipulação de dataframes
import numpy as np                                                # Manipulação de séries numéricas
import matplotlib.pyplot as plt                                   # Para manipulação/criação de gráficos
import os                                                         # Manipular arquivos em diretórios de sistema
from pathlib import Path                                          # Manipular diretórios

## ANTES DE CONTINUAR, ALGUMAS DICAS:

##### DICAS DE TIPOS DE GRÁFICOS

*   Gráfico de barras (bar)
*   Gráfico de linhas (line)
*   Gráfico de pizza (pie)
*   Gráfico de dispersão (Scatterplot)

Para definir eixos:

*   xlabel = 'nome do eixo X'
*   ylabel = 'nome do eixo Y'

Para definir títulos

*   title = 'nome do título'

Para mudar cor de um objeto

*   color = 'nome da cor (em inglês)'


### DEFINIÇÃO DE FUNÇÕES

#### FUNÇÃO PARA REALIZAR DOWNLOAD DE ARQUIVOS

In [6]:
def download_dados (url_arquivo, nome_arquivo):
  requisicao = requests.get(url_arquivo)  # Realiza a requisição na URL
  conteudo = requisicao.content           # Salva o conteúdo do arquivo na variável "conteúdo"
  arquivo = open(nome_arquivo, 'wb')      # Abre arquivo com o nome enviado na função com modo de escrita binária
  arquivo.write(conteudo)                 # Grava o conteúdo da variável "conteudo" no arquivo
  arquivo.close()                         # Fecha o arquivo

### DEFINIÇÃO DE VARIÁVEIS (EXEMPLO)

In [8]:
url_ordem_compra = 'https://datalakeaula149820.blob.core.windows.net/datalake/raw-zone/olist/olist_orders_dataset/olist_orders_dataset.csv'
nome_arquivo_ordem_compra = 'olist_orders_dataset.csv'

In [9]:
url_ordem_reviews = 'https://datalakeaula149820.blob.core.windows.net/datalake/raw-zone/olist/olist_order_reviews_dataset/olist_order_reviews_dataset.csv'
nome_arquivo_ordem_reviews = 'olist_order_reviews_dataset.csv'

In [10]:
url_clientes = 'https://datalakeaula149820.blob.core.windows.net/datalake/raw-zone/olist/olist_customers_dataset/olist_customers_dataset.csv'
nome_arquivo_clientes = 'olist_customers_dataset.csv'

### REALIZA DOWNLOAD DOS ARQUIVOS

In [12]:
download_dados(url_ordem_compra, nome_arquivo_ordem_compra)
download_dados(url_ordem_reviews, nome_arquivo_ordem_reviews)
download_dados(url_clientes, nome_arquivo_clientes)

### LEITURA DOS ARQUIVOS (USANDO PYTHON/PANDAS)

In [ ]:
df_ordem_compra = pandas.read_csv(nome_arquivo_ordem_compra)

#### EXIBINDO AS COLUNAS DOS ARQUIVOS

In [ ]:
df_ordem_compra.columns

#### EXIBINDO OS METADADOS DO ARQUIVO

In [ ]:
df_ordem_compra.info()

#### EXIBINDO UMA AMOSTRA DO ARQUIVO

In [ ]:
df_ordem_compra.head(n=10)

#### MODIFICANDO A ESTRUTURA DE ARQUIVO

##### EXEMPLO 1: ALTERANDO O TIPO DE DADO DE UMA(S) COLUNA(S)

In [ ]:
df_ordem_compra['order_id'] = df_ordem_compra['order_id'].astype('string')
df_ordem_compra['customer_id'] = df_ordem_compra['customer_id'].astype('string')
df_ordem_compra['order_status'] = df_ordem_compra['order_status'].astype('string')
df_ordem_compra['order_purchase_timestamp'] = df_ordem_compra['order_purchase_timestamp'].astype('datetime64')
df_ordem_compra['order_approved_at'] = df_ordem_compra['order_approved_at'].astype('datetime64')
df_ordem_compra['order_delivered_carrier_date'] = df_ordem_compra['order_delivered_carrier_date'].astype('datetime64')
df_ordem_compra['order_delivered_customer_date'] = df_ordem_compra['order_delivered_customer_date'].astype('datetime64')
df_ordem_compra['order_estimated_delivery_date'] = df_ordem_compra['order_estimated_delivery_date'].astype('datetime64')


###### VALIDANDO AS ALTERAÇÕES

###### EXIBINDO AS ALTERAÇÕES NOS METADADOS

In [ ]:
df_ordem_compra.info()

###### EXIBINDO UMA AMOSTRA DO ARQUIVO

In [ ]:
df_ordem_compra.head(10)

##### EXEMPLO 2: ALTERANDO O SCHEMA DO ARQUIVO USANDO UM DICIONÁRIO

In [ ]:
schema_ordem_compra = {
    "order_id" : "string",
    "customer_id" : "string",
    "order_status" : "string",
    "order_purchase_timestamp" : "datetime64",
    "order_approved_at" : "datetime64",
    "order_delivered_carrier_date" : "datetime64",
    "order_delivered_customer_date" : "datetime64",
    "order_estimated_delivery_date" : "datetime64"
}

In [ ]:
df_ordem_compra = df_ordem_compra.astype(schema_ordem_compra)

###### VALIDANDO AS ALTERAÇÕES

In [ ]:
df_ordem_compra.info()

###### EXIBINDO UMA AMOSTRA DO ARQUIVO

In [ ]:
df_ordem_compra.head(10)

#### MANIPULAÇÃO NO(S) ARQUIVO(S)

##### IDENTIFICANDO A QUANTIDADE DE COMPRAS

In [ ]:
df_ordem_compra['order_id'].count()

##### IDENTIFICANDO A QUANTIDADE DISTINTA DE USUÁRIOS

In [ ]:
df_ordem_compra['customer_id'].nunique()

##### IDENTIFICANDO A ORDEM DE COMPRA MAIS ANTIGA E MAIS RECENTE

##### EXEMPLO 1: USANDO NUMPY E AGGREGATE

In [ ]:
df_ordem_compra.agg({'order_purchase_timestamp' : [np.min, np.max]})

##### EXEMPLO 2: USANDO MIN E MAX SEPARADAMENTE

In [ ]:
df_ordem_compra['order_purchase_timestamp'].min() # OU min(df_ordem_compra['order_purchase_timestamp'])

In [ ]:
df_ordem_compra['order_purchase_timestamp'].max() # OU max(df_ordem_compra['order_purchase_timestamp'])

##### EXEMPLO 3: USANDO A FUNÇÃO DESCRIBE

In [ ]:
df_ordem_compra.describe()

##### ANALISANDO A DISTRIBUIÇÃO DE COMPRAS POR ANO/MÊS (GRÁFICO DE BARRAS)

In [ ]:
df_ordem_compra[['order_id']] \
  .groupby([df_ordem_compra['order_purchase_timestamp'].dt.year, df_ordem_compra['order_purchase_timestamp'].dt.month]) \
  .agg(['count']).rename(columns={'order_id' : 'compras'}) \
  .plot(xlabel = 'Ano/Mês', ylabel='Compras', title='Compras por ano', color="green", kind='bar')

#### GRAVANDO UM ARQUIVO USANDO O MÉTODO PANDAS

### LEITURA DOS ARQUIVOS (USANDO PYTHON/SPARK)

#### CRIAÇÃO DA APLICAÇÃO SPARK

In [13]:
spark = SparkSession.builder.getOrCreate() # Constroi (ou obtem de um ambiente criado) uma aplicação
sqlcontext = SQLContext(spark)                     # Cria um contexto Spark para trabalhar com funções SQL
sparkcontext = spark.sparkContext          # Cria um contexto Spark para manipular dados de forma distribuiída

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [14]:
dfs_ordem_compra = spark.read.csv(nome_arquivo_ordem_compra, header=True, inferSchema=True)
dfs_ordem_reviews = spark.read.csv(nome_arquivo_ordem_reviews, header=True, inferSchema=True)
dfs_clientes = spark.read.csv(nome_arquivo_clientes, header=True, inferSchema=True)

#### EXIBINDO AS COLUNAS DOS ARQUIVOS

In [15]:
dfs_ordem_compra.columns

['order_id',
 'customer_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date']

In [16]:
dfs_ordem_reviews.columns

['review_id',
 'order_id',
 'review_score',
 'review_comment_title',
 'review_comment_message',
 'review_creation_date',
 'review_answer_timestamp']

In [17]:
dfs_clientes.columns

['customer_id',
 'customer_unique_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state']

#### EXIBINDO OS METADADOS DO ARQUIVO

In [18]:
dfs_ordem_compra.summary().show()

+-------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|summary|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+-------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|  count|               99441|               99441|       99441|                   99441|              99281|                       97658|                        96476|                        99441|
|   mean|                null|                null|        null|                    null|               null|                        null|                         null|                         null|
| std

In [22]:
dfs_ordem_reviews.summary().show()

+-------+------------------+--------------------+--------------------+--------------------+----------------------+--------------------+-----------------------+
|summary|         review_id|            order_id|        review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+-------+------------------+--------------------+--------------------+--------------------+----------------------+--------------------+-----------------------+
|  count|            105188|              102859|              102692|               12176|                 41868|               96025|                  96002|
|   mean|               4.5|                 0.0|   4.071667849964501|3.155449396525236...|  1.111111111111111...|                null|                   null|
| stddev|0.7071067811865476|                 0.0|  1.3866488774346788|5.616554832455847E11|              Infinity|                null|                   null|
|    min|                 "|            

In [23]:
dfs_clientes.summary().show()

+-------+--------------------+--------------------+------------------------+-------------------+--------------+
|summary|         customer_id|  customer_unique_id|customer_zip_code_prefix|      customer_city|customer_state|
+-------+--------------------+--------------------+------------------------+-------------------+--------------+
|  count|               99441|               99441|                   99441|              99441|         99441|
|   mean|                null|                null|       35137.47458291851|               null|          null|
| stddev|                null|                null|      29797.938996206158|               null|          null|
|    min|00012a2ce6f8dcda2...|0000366f3b9a7992b...|                    1003|abadia dos dourados|            AC|
|    25%|                null|                null|                   11347|               null|          null|
|    50%|                null|                null|                   24416|               null|        

#### EXIBINDO UMA AMOSTRA DO ARQUIVO

In [19]:
dfs_ordem_compra.show(n=5)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [20]:
dfs_ordem_reviews.show(n=5)

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                null|                  null| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                null|                  null| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                null|                  null| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|                null|  Recebi bem antes ...| 2017-04-21 00:00:00|   

In [21]:
dfs_clientes.show(n=5)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+
only showing top 5 rows



#### EXIBINDO O SCHEMA DOS DADOS

In [ ]:
dfs_ordem_compra.printSchema()

In [ ]:
dfs_ordem_reviews.printSchema()

In [ ]:
dfs_clientes.printSchema()

#### CRIAR UMA VISÃO PARA TRABALHAR COM DATAFRAMES USANDO LINGUAGEM SQL

In [32]:
dfs_ordem_reviews.createOrReplaceTempView('OrdemReviewTemp')

In [33]:
dfs_ordem_compra.createOrReplaceTempView('OrdemCompraTemp')

In [35]:
dfs_clientes.createOrReplaceTempView('OrdemClientesTemp')

In [40]:
resultado1 = sqlcontext.sql("""
SELECT *
FROM  OrdemReviewTemp
""")

In [46]:
resultado1.show(3)

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                null|                  null| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                null|                  null| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                null|                  null| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+---

In [42]:
resultado2 = sqlcontext.sql("""
SELECT *
FROM  OrdemCompraTemp
""")

In [48]:
resultado2.show(3)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [44]:
resultado3 = sqlcontext.sql("""
SELECT *
FROM  OrdemClientesTemp
""")

In [47]:
resultado3.show(3)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+
only showing top 3 rows



In [61]:
#Consulta, media e total de nota por Estados
resultado4 = sqlcontext.sql("""
SELECT 
    AVG(review_score), COUNT(review_id), customer_state
FROM  
    OrdemReviewTemp 
INNER JOIN 
    OrdemCompraTemp
ON OrdemReviewTemp.order_id = OrdemCompraTemp.order_id
INNER JOIN 
    OrdemClientesTemp
ON OrdemCompraTemp.customer_id = OrdemClientesTemp.customer_id
GROUP BY customer_state
""")

In [62]:
resultado4.show()

+------------------+----------------+--------------+
| avg(review_score)|count(review_id)|customer_state|
+------------------+----------------+--------------+
| 4.055875102711586|            3651|            SC|
| 4.055335968379446|             253|            RO|
| 3.907258064516129|             496|            PI|
| 4.154362416107382|             149|            AM|
| 3.608695652173913|              46|            RR|
| 4.026018654884634|            2037|            GO|
|               4.1|             280|            TO|
| 4.087912087912088|             910|            MT|
| 4.160578549812948|           41967|            SP|
| 4.009789525208028|            2043|            ES|
| 4.007448789571694|             537|            PB|
| 4.125658253132377|            5507|            RS|
| 4.115702479338843|             726|            MS|
|3.7314148681055155|             417|            AL|
| 4.121049034683068|           11706|            MG|
| 3.827902240325866|             982|         

#### GRAVANDO UM ARQUIVO USANDO O SPARK

In [ ]:
dfs_ordem_compra.coalesce(1).write.mode("overwrite").format("csv").options(header=True, delimiter=";").save("pasta")

## ANÁLISE DE SENTIMENTOS

### DEFINIÇÃO DE PARÂMETROS PARA USAR A API

In [63]:
chave_api = "7fb9a30163934e25a408db983ecae564" # COLOQUE AQUI A SUA CHAVE
endpoint_api = "https://puc-txt-ia149820.cognitiveservices.azure.com/" # COLOQUE AQUI O SEU ENDPOINT

### CRIAÇÃO DE FUNÇÃO PARA CHAMAR A API

In [64]:
def criar_cliente():
    criar_cliente_analise = TextAnalyticsClient(
        endpoint=endpoint_api,
        credential=AzureKeyCredential(chave_api)
    )
    return criar_cliente_analise

### FUNÇÃO PARA ANALISAR SENTIMENTOS

In [70]:
documento = ["Eu odeio amar", "eu amo odiar"]

In [66]:
cliente = criar_cliente()

In [71]:
cliente.analyze_sentiment(documento)

[AnalyzeSentimentResult(id=0, sentiment=negative, warnings=[], statistics=None, confidence_scores=SentimentConfidenceScores(positive=0.1, neutral=0.04, negative=0.87), sentences=[SentenceSentiment(text=Eu odeio amar, sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.1, neutral=0.04, negative=0.87), length=13, offset=0, mined_opinions=[])], is_error=False),
 AnalyzeSentimentResult(id=1, sentiment=positive, warnings=[], statistics=None, confidence_scores=SentimentConfidenceScores(positive=0.51, neutral=0.03, negative=0.46), sentences=[SentenceSentiment(text=eu amo odiar, sentiment=positive, confidence_scores=SentimentConfidenceScores(positive=0.51, neutral=0.03, negative=0.46), length=12, offset=0, mined_opinions=[])], is_error=False)]

In [ ]:
def funcao_analisa_sentimentos(cliente, id, documento):
    response = cliente.analyze_sentiment(documents=documento)[0]

    df_resultado = sqlcontext.createDataFrame([( str(id), str(response.sentiment), str(response.confidence_scores.positive), str(response.confidence_scores.negative), str(response.confidence_scores.neutral))], ("order_id", "sentimento", "score_positivo","score_negativo","score_neutro"))
    return df_resultado

### FUNÇÃO PARA ARMAZENAR DADOS NO AZURE DATALAKE STORAGE GEN2

#### VARIÁVEL COM A CADEIA DE CONEXÃO DO SERVIÇO AZURE STORAGE ACCOUNT

In [ ]:
endpoint_azure = ""

In [ ]:
def carrega_arquivo_blob(local_destino, nome_arquivo, caminho_destino):
    
    # PARA BUSCAR O ENDPOINT DA SUA CONTA, ACESSE O SERVIÇO "CONTAS DE ARMAZENAMENTO"
    # NO PAINEL A ESQUERDA, NO MENU "SEGURANÇA + REDE", SELECIONE A OPÇÃO "CHAVES DE ACESSO"
    # CLIQUE EM "MOSTRAR AS CHAVES" E COPIE A CADEIA DE CONEXÃO DE UMA DAS CHAVES (1 OU 2)
    # COLE TODA A INSTRUÇÃO COMO VALOR DA VARIÁVEL ABAIXO
    conexao_blob = ""
    blob_service_client = BlobServiceClient.from_connection_string(conexao_blob)
    blob_client = blob_service_client.get_blob_client(container=local_destino, blob=nome_arquivo)
    try:
        with open(caminho_destino, "rb") as data:
            blob_client.upload_blob(data, overwrite=True)
            print("Arquivo carregado com sucesso")
    except NameError:
        print("Erro")

In [ ]:
localdestino = "" # Mude para o nome do Contêiner da sua conta (Exemplo: datalake/raw-zone/olist_ecommerce/)
caminho_pasta_local = "orders_reviews"
arquivos = os.listdir(caminho_pasta_local)

for arquivo_origem in arquivos:
    if (arquivo_origem.endswith('.csv')):
        caminho_destino = os.path.join(caminho_pasta_local, arquivo_origem)
        arquivo = ""
        carrega_arquivo_blob (localdestino, arquivo, caminho_destino)